In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

In [2]:
bdname='sinesearch_db'
bduser = 'postgres'
bdpassword = 'nef441'
bdhost = '51.79.69.179'
bdport = 5432

db = PostgresqlExtDatabase(bdname, user=bduser, password=bdpassword,
                           host=bdhost, port=bdport)

In [12]:
db.rollback()

In [6]:
class Users(Model):
    tel_id=IntegerField(index=True)
    name = TextField()
    nicname = TextField(null=True)
    ustatus=IntegerField(default=0)
    age=IntegerField(default=20)
    sex=IntegerField(default=0)
    ganres=JSONField(default={})
    info=JSONField(default={})
    last_visit=DateField(null=True)

    cms=IntegerField(default=0)
    cfid=IntegerField(default=0)
    cfmes=IntegerField(default=0)
    ctmes=IntegerField(default=0)
    liked=ArrayField(IntegerField,default=[])
    viewed=ArrayField(IntegerField,default=[])
    disliked=ArrayField(IntegerField,default=[])
    predict_films=ArrayField(IntegerField,default=[])
    selections=JSONField(default={})

    class Meta:
        database = db
        db_table='Users'

class Films(Model):
    film_id=IntegerField(unique=True,index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube=TextField(default="")
    stars = FloatField(default=0)
    info=JSONField(default={})
    level=IntegerField(default=0)
    selections=ArrayField(IntegerField,default=[])
    year=IntegerField(default=2000)

    ganres=ArrayField(IntegerField,default=[])
    meanage=FloatField(default=20)
    sex=FloatField(default=0.5)

    likes=IntegerField(default=10)
    dislikes=IntegerField(default=10)
    shit=IntegerField(default=0)
    errors=IntegerField(default=0)
    opening=IntegerField(default=0)
    treilers=IntegerField(default=0)

    class Meta:
        database = db
        db_table='Films'

class Selections(Model):
    name = TextField()
    user_id = IntegerField(default=0)
    stars = FloatField(default=0)
    films = ArrayField(IntegerField,default=[])

    class Meta:
        database = db
        db_table='Selections'

class Messages(Model):
    mes_id = TextField(unique=True)
    text = TextField(default="")
    reply_markup = JSONField(default={})
    user = IntegerField(default=0)
    btime = DateField(null=True)

    class Meta:
        database = db
        db_table='Messages'

class Config(Model):
    name = TextField()
    value = TextField()
    json=JSONField(default={})

    class Meta:
        database = db
        db_table='Config'

class Dataset(Model):
    user_value = JSONField()
    film_value = JSONField()
    result = IntegerField(default=0)
    data = DateField()

    class Meta:
        database = db
        db_table='Dataset'

In [11]:
Films.select().count()-26374

18

In [8]:
with open("collections","r") as f:
    collections=json.load(f)

In [ ]:
col=[]
for i , c in tqdm(enumerate(collections)):
    col.append({
         "name":c,
         "films":[int(k) for k in collections[c]]
     })
    if i % 100 ==0:
        Selections.insert_many(col).execute()
        col=[]
Selections.insert_many(col).execute()

In [10]:
fms=[]
db.rollback()
with open("films.jsonlines","r") as f:
    for k, line in tqdm(enumerate(f.readlines())):
        film = json.loads(line)
        info=film.copy()
        if "Жанр" not in film:
            film["Жанр"]=""
        
        
        comp_film={
            "name":info["name"],
            "discr":info["discr"],
            "img":info["img"],
            "youtube":info["youtube"],
            "film_id":info["film_id"],   
            "stars":0,
            "ganres":[gange.index(g) for g in film["Жанр"].replace(" ","").replace(".","").split(",")]
        }
        try:
            comp_film["year"]=int(re.search(r"\(\d+\)",info["name"])[0][1:-1])
            comp_film["name"]=comp_film["name"].replace(re.search(r"\(\d+\)",info["name"])[0],"")
        except:
            print(comp_film["name"])
            comp_film["year"]=2000
#             break
            
        if "Рейтинг" in info:
            comp_film["stars"] = float(info["Рейтинг"].replace(" ","").replace(",",".").replace("%",""))/10
        del info["name"]
        del info["discr"]
        del info["img"]
        del info["youtube"]
        del info["film_id"]
#         info["Год выпуска"]=comp_film["year"]
        del info["data"]

        
        comp_film["selections"]=[]
        comp_film["level"]=4
        
            
        comp_film["info"]=info
#         break
        
        fms.append(comp_film)
        if k % 500 == 0:
            Films.insert_many(fms).on_conflict_ignore().execute()   
            fms=[]
Films.insert_many(fms).on_conflict_ignore().execute()

In [ ]:
len(Films.select().where(Films.level==1).count().execute())

In [ ]:
with open("films","w") as f:
    json.dump(fms,f)

**Ganre**

In [8]:
with open("gange","r") as f:
    gange = json.load(f)

In [ ]:
similars={}
for g in gange:
    similars[g]={}
no=0
with open("films.jsonlines","r") as f:
    for k, line in tqdm(enumerate(f.readlines())):
        film = json.loads(line)
#         print(film)
#         break
        if not "Жанр" in film:
            no+=1
            continue
        for g in film["Жанр"].replace(" ","").replace(".","").split(","):
            for g2 in film["Жанр"].replace(" ","").replace(".","").split(","):
                if g2 in similars[g]:
                    similars[g][g2]+=1
                else:
                    similars[g][g2]=1
        
print(no)        

In [ ]:
variants={}
for s in similars:
    top = sorted(similars[s],key=lambda k:similars[s][k])[-5:-1]
    variants[s] = top

In [ ]:
bests={}
for v in variants:
    with open("films.jsonlines","r") as f:
        for k, line in tqdm(enumerate(f.readlines())):
            film = json.loads(line)
            if not "Жанр" in film:
                no+=1
                continue
            cont=False
            for g in film["Жанр"].replace(" ","").replace(".","").split(","):
                if g not in variants[v]:
                    cont=True
                    break
            if cont:
                continue
                    
            if not v in bests:
                bests[v]=[]
            bests[v].append(film["film_id"])
            print(fms[film["film_id"]+])

In [ ]:
for b in bests:            
    bests[b]=sorted(bests[b],key = lambda k: fms[str(k)]["stars"]+len(fms[str(k)]["selections"])/5)[-10:]

In [ ]:
bests["Боевик"][2]

In [ ]:
sorted(fms,key=lambda k: len(fms[k]["selections"]))[-10:]

In [ ]:
{
    "name":1,
    "ere":"wewew"
}

In [ ]:
variants